# scikit learn (sklearn)

Scikit-learn es una librería de código abierto para Python, que implementa una variedad de algoritmos de Minería de Datos, pre-procesamiento, referencias cruzadas y visualización usando una interfaz unificada.

## Extracción de características
Utilizamos el módulo **sklearn.feature_extraction** para convertir documentos de texto (y de otros tipos) al espacio de vectores.
Este módulo permite:
    * Tokenizar strings y dar un identificador numérico a cada posible token, por ejemplo utilizando espacios y signos de puntuación como separadores
    * Conteo de frecuencias de ocurrencia de tokens en cada documento
    * Normalización y pesado para dar poco peso a los tokens que aparecen en la mayoría de los documentos

Un corpus de documentos entonces es representado como una matriz, donde cada fila corresponde a un documento y cada columna a la presencia o no de un token. Este proceso se conoce como **vectorización** y la estrategia de tokenizar, contar y normalizar como **bag of words** (bolsa de palabras)



## Matrices dispersas
Como la mayoría de los documentos suelen utilizar un subconjunto muy pequeño de las palabras utilizadas en el corpus, la matriz resultante tendrá muchos valores de características que son ceros.

Por ejemplo, una colección de 10.000 documentos de texto corto (como correos electrónicos) usará un vocabulario con un tamaño del orden de 100.000 palabras únicas en total, mientras que cada documento usará de 100 a 1000 palabras únicas individualmente.

Para poder almacenar dicha matriz en memoria pero también para acelerar las operaciones sobre ella, las implementaciones usan típicamente una representación dispersa tal como las implementaciones disponibles en el paquete **scipy.sparse**

## CountVectorizer
La tokenización y el conteo de palabras en sklearn los realiza la clase CountVectorizer

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

Este modelo tiene muchos parámetros, pero los valores por defecto son generalmente aceptables

In [2]:
vectorizer = CountVectorizer()
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

El **analyzer** indica qué se considera un token: caracteres o palabras {‘word’, ‘char’, ‘char_wb’} . La opción 'char_wb' crea  n-gramas de caracteres solo desde el texto dentro de los límites de la palabra; Los n-grams en los bordes de las palabras se rellenan con espacio.
Si se pasa una función en este parámetro, se utiliza para extraer la secuencia de características de la entrada en bruto sin procesar.

El parámetro **binary** se usa si no queremos contar la cantidad de ocurrencias de cada palabra, sino solo su existencia (modelo binario)

**lowercase** se indica si el texto debe convertirse a minúsculas antes de tokenizar

**stopwords** recibe el string ‘english’, una lista de palabras, or None (default)

**token_pattern** recibe una expresión regular que define lo que significa un token, y solo es usada si se utiliza el analizador de palabras ('word'). La expresión por defecto selecciona tokens con 2 o más caracteres alfanuméricos (los signos de puntuación se ignoran y se tratan siempre como separadores de palabras)

**max_df**  es un número entre 0 y 1 (por defecto es 1) que indica que al construir el vocabulario se ignoren los términos que tienen una frecuencia de documento estrictamente mayor que el umbral dado (stopwords específicas del corpus). Si se da un número no entero, representa una proporción de documentos. Si se da un número entero, representa cantidad. Este parámetro se ignora si el vocabulario no es None 

De forma similar, **min_df** indica que se ignoren los términos que tienen una frecuencia strictamente menor al valor dado. Conocido como *cut-off* en la literatura

**ngram_range** espera una tupla (min_n, max_n) con los valores mínimo y máximo de ngramas a extraer. Se usan todos los valores de n tal que *min_n <= n <= max_n*.

**preprocessor** puede recibir una función que sobreescribe la etapa de preprocessamiento, pero mantiene la tokenización y la generación de ngramas 

**tokenizer** puede recibir una función que sobrescribe la tokenización de los strings, pero preserva el preprocesamiento y la generación de ngramas. Solo se aplica si *analyzer == 'word'*

**max_features** Si es un número entero, distinto de None, se construye un vocabulario que solo considera las principales max_features, ordenadas por frecuencia. Se ignora si el vocabulario no es None

**vocabulary** Espera opcionalmente un Mapping (por ejemplo un dict) donde las claves son términos y los valores son los índices en la matriz de características, o un iterable de terminos. Si no se proporciona este parámetro, el vocabulario se determina a partir de los documentos. Los índices en el mapping no deben repetirse y deben ser consecutivos (no debe haber ningún "hueco" entre 0 y el índice más largo)


In [5]:
corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]
X = vectorizer.fit_transform(corpus)
X   

<4x9 sparse matrix of type '<class 'numpy.int64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [4]:
print(X)

  (0, 1)	1
  (0, 2)	1
  (0, 6)	1
  (0, 3)	1
  (0, 8)	1
  (1, 5)	2
  (1, 1)	1
  (1, 6)	1
  (1, 3)	1
  (1, 8)	1
  (2, 4)	1
  (2, 7)	1
  (2, 0)	1
  (2, 6)	1
  (3, 1)	1
  (3, 2)	1
  (3, 6)	1
  (3, 3)	1
  (3, 8)	1


La función especifica que realiza la tokenización puede invocarse explícitamente. Notar que solo se consideran palabras de al menos dos letras

In [9]:
analyze = vectorizer.build_analyzer()
analyze('This is a text document to analyze')

['this', 'is', 'text', 'document', 'to', 'analyze']

A cada término encontrado por el analizador se le asigna un identificador entero único que corresponde a una columna en la matriz resultante. Esta interpretación de columnas puede obtenerse de la siguiente manera:

In [10]:
features = vectorizer.get_feature_names()
features

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

In [11]:
X.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 2, 1, 0, 1],
       [1, 0, 0, 0, 1, 0, 1, 1, 0],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

Notar que el primer documento y el último poseen vectores idénticos, ya que utilizan las mismas palabras (aunque en diferente orden)

El mapeo inverso puede realizarse inspeccionando la variable vocabulary_ del vectorizer:

In [12]:
vectorizer.vocabulary_.get('document')

1

Las palabras que no se observaron durante el entrenamiento, se ignoran en futuras llamadas al método transform

In [14]:
t =vectorizer.transform(["This is a completely new document that was not observed previously in this dataset"]).toarray()
t
dict(zip(features, t[0]))

{'and': 0,
 'document': 1,
 'first': 0,
 'is': 1,
 'one': 0,
 'second': 0,
 'the': 0,
 'third': 0,
 'this': 2}

In [15]:
vectorizer.transform(["Something completely new"]).toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

Para evitar perder la información del orden de las palabras (como ocurre con el primer documento y el último), podemos extraer bigramas, además de los 1-gramas (palabras individuales)

In [16]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), 
                                    token_pattern=r'\b\w+\b', 
                                    min_df=1)
analyze = bigram_vectorizer.build_analyzer()
analyze('Bi-grams are cool!') 

['bi', 'grams', 'are', 'cool', 'bi grams', 'grams are', 'are cool']

Nota: en Python la r delante de un string indica que las '\' dentro del string se tomen como parte de la expresión regular, y no para escapar el caracter siguiente. Por lo tanto la expresión **r'\b\w+\b'** es quivalente a **'\\\\b\\\\w+\\\\b'**

In [18]:
char_bigram_vect = CountVectorizer(ngram_range=(3, 3), analyzer='char_wb')
analyze2 = char_bigram_vect.build_analyzer()
analyze2('Bigrams are cool!') 

[' bi',
 'big',
 'igr',
 'gra',
 'ram',
 'ams',
 'ms ',
 ' ar',
 'are',
 're ',
 ' co',
 'coo',
 'ool',
 'ol!',
 'l! ']

El vocabulario extraido con este vectorizador es mucho más grande y permite resolver ambigüedades codificadas en las posiciones locales

In [21]:
X_2 = bigram_vectorizer.fit_transform(corpus)
X_2

<4x21 sparse matrix of type '<class 'numpy.int64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [24]:
X_2 = bigram_vectorizer.fit_transform(corpus).toarray()
bigram_vectorizer.get_feature_names()



['and',
 'and the',
 'document',
 'first',
 'first document',
 'is',
 'is the',
 'is this',
 'one',
 'second',
 'second document',
 'second second',
 'the',
 'the first',
 'the second',
 'the third',
 'third',
 'third one',
 'this',
 'this is',
 'this the']

In [25]:
X_2

array([[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1]],
      dtype=int64)

En particular, la forma interrogativa "Is this" sólo está presente en el último documento

In [26]:
feature_index = bigram_vectorizer.vocabulary_.get('is this')
print(X_2[:,feature_index]) #obtengo toda la columna indicada por feature_index

[0 0 0 1]


# Tf-idf

En algunos corpus pueden existir palabras que estén muy presentes y por lo tanto aporten poca información significativa acerca de los contenidos del documento.

La transformación tf/idf se utiliza para pesar los términos inversamente a la cantidad de documentos en los que aparece la palabra

$$\text{tf-idf(t,d)}=\text{tf(t,d)} \times \text{idf(t, D)}$$
$${\displaystyle \mathrm {idf} (t,D)=\log {\frac {|D|}{|\{d\in D:t\in d\}|}}}$$

 - **TfidfTransformer**: Normaliza una matriz de cuentas de palabras aplicando TF/IDF
 - **TfidfVectorizer**: Aplica sobre el corpus directamente

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False, norm='l1')
transformer   

TfidfTransformer(norm='l1', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

**norm**: 'l1', 'l2' o None, optional
Se utiliza para normalizar los vectores de términos. Usar None para no normalizar.

**use_idf**: boolean, default=True
Habilita el uso de frecuencias inversas

**smooth_idf**: boolean, default=True
Suaviza los pesos idf agregando 1 a las frecuencias de los documentos, como si existiera un documento extra que contiene el término exactamente 1 vez. Evita divisiones por cero. 

**sublinear_tf**: boolean, default=False
Aplica escalado tf sublineal, es decir, reemplaza tf con 1 + log(tf)




Veamos el siguiente ejemplo en el que el primer término se encuentra presente en todos los documentos por lo que no es representativo. Los otros dos términos se encuentran en menos del 50% de los documentos y por lo tanto podrían ser representativos

In [28]:
counts = [[3, 0, 1],
          [2, 0, 0],
          [3, 0, 0],
          [4, 0, 0],
          [3, 2, 0],
          [3, 0, 2]]

tfidf = transformer.fit_transform(counts)
tfidf                         
  

<6x3 sparse matrix of type '<class 'numpy.float64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [29]:
tfidf.toarray()

array([[0.5883954 , 0.        , 0.4116046 ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.34950701, 0.65049299, 0.        ],
       [0.41682734, 0.        , 0.58317266]])

Cada fila está normalizada para que sume 1 aplicando la norma euclídea (L1)

In [30]:
transformer2 = TfidfTransformer()
transformer2

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

En este caso, smooth_idf es True, por lo que se agrega 1 al numerador y al denominador como si existira un documento extra que tiene el término exactamente 1 vez

In [31]:
transformer2.fit_transform(counts).toarray()

array([[0.85151335, 0.        , 0.52433293],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        ],
       [0.55422893, 0.83236428, 0.        ],
       [0.63035731, 0.        , 0.77630514]])

Los pesos de cada término calculado por el método fit se almacenan en un atributo

In [32]:
transformer.idf_

array([1.        , 2.79175947, 2.09861229])

In [33]:
transformer2.idf_ 

array([1.        , 2.25276297, 1.84729786])

Como tf/idf se utiliza frecuentemente para texto, existe otra clase llamada **TfidfVectorizer** que combina todas las opciones de CountVectorizer y TfidfTranformer en un único modelo

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer3 = TfidfVectorizer()
X_3 = vectorizer3.fit_transform(corpus)
X_3

<4x9 sparse matrix of type '<class 'numpy.float64'>'
	with 19 stored elements in Compressed Sparse Row format>

In [35]:
X_3.toarray()

array([[0.        , 0.43877674, 0.54197657, 0.43877674, 0.        ,
        0.        , 0.35872874, 0.        , 0.43877674],
       [0.        , 0.27230147, 0.        , 0.27230147, 0.        ,
        0.85322574, 0.22262429, 0.        , 0.27230147],
       [0.55280532, 0.        , 0.        , 0.        , 0.55280532,
        0.        , 0.28847675, 0.55280532, 0.        ],
       [0.        , 0.43877674, 0.54197657, 0.43877674, 0.        ,
        0.        , 0.35872874, 0.        , 0.43877674]])

# Selección de características

Un ejemplo de selección de características es la Información Mutua (MI). Esta métrica calculada para dos variables aleatorias es un número no negativo que mide la dependencia entre ambas variables. Es igual a 0 si y solo si las dos variables son independientes y los valores mayores significan dependencia.

$${\displaystyle I(X;Y)=\sum _{y\in Y}\sum _{x\in X}p(x,y)\log {\left({\frac {p(x,y)}{p(x)\,p(y)}}\right)}}$$ (X son las características e Y las clases)
$${\displaystyle I(Atributo;Clase)=H(Clase) - H(Clase | Atributo)}$$

In [36]:
from sklearn.feature_selection import mutual_info_classif

corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
]
clases = ['A','B','C','A']
cv = CountVectorizer()
X_vec = cv.fit_transform(corpus)
res = dict(zip(cv.get_feature_names(),
               mutual_info_classif(X_vec, clases, discrete_features=True)
               ))

#Imprimimos ordenados por MI
from operator import itemgetter
sorted(res.items(), key=itemgetter(1))

[('the', 5.551115123125783e-17),
 ('and', 0.5623351446188083),
 ('document', 0.5623351446188083),
 ('is', 0.5623351446188083),
 ('one', 0.5623351446188083),
 ('second', 0.5623351446188083),
 ('third', 0.5623351446188083),
 ('this', 0.5623351446188083),
 ('first', 0.6931471805599452)]